In [1]:
# number_of_chunks
# "Camel":      3 (each chunk = 15,297 | 15,296)
# "CloudStack": 8 (each chunk =  3,590 |  3,589)
# "Geode":      4 (each chunk =  | )
# "Hbase":      4 (each chunk =  | )

# bug_localization_accuracy: [0.4, 0.5, 0.6, 0.7, 0.8]

# Set Variables

In [2]:
which_chunck = 4
top_k = 20

# -------- method_way
method_way = "c" # a, b, c

if method_way == "a":
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold =    [1]

elif method_way == "b":
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold = [
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]

else: # method_way == "c"
    bug_localization_accuracy = [0.4]
    knn_accuracy_threshold = [
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]

# -------- dataset
# software_name = "Camel"
software_name = "CloudStack"
# software_name = "Geode"
# software_name = "Hbase"

In [3]:
print("bla: ", bug_localization_accuracy)
print("th:  ", knn_accuracy_threshold)

bla:  [0.4]
th:   [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


In [4]:
if software_name == "Camel":
    dataset_file_name =    "Camel_Bugs - v.01"
    dataset_file_name_ts = "Camel_TimeSeries - v.01"
    
elif software_name == "CloudStack":
    dataset_file_name =    "CloudStack_Bugs - v.01"
    dataset_file_name_ts = "CloudStack_TimeSeries - v.01"
    
elif software_name == "Geode":
    dataset_file_name =    "Geode_Bug - v.01"
    dataset_file_name_ts = "Geode_TimeSeries - v.01"
    
else:
    dataset_file_name =    "Hbase_Bugs - v.03"
    dataset_file_name_ts = "Hbase_TimeSeries - v.03"

# Google Colab

In [5]:
!pip install enlighten
!pip install --upgrade statsmodels 
!pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# load data from google drive
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
!ls "/content/gdrive/My Drive/"

Mounted at /content/gdrive
'Colab Notebooks'


In [7]:
# project folder path
project_folder = "gdrive/MyDrive/Colab Notebooks/paper/"

# data folder path
data_folder = "00- My Data/two-phased method/"

# dataset folder path
dataset_folder = software_name + "/"

# output folder path
output_folder = "01- Jupyter Notebook/two-phased method/00. Output/"

# Libs

In [8]:
import enlighten
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller
from sklearn.neighbors import KNeighborsClassifier

In [9]:
import warnings
warnings.filterwarnings("ignore")

# Config

In [10]:
mypaths = {
    "data": {
        "dataset_bugs_bla_chunk":      project_folder + data_folder + dataset_folder + "second/k={}/bla={}/"      + "{} _ k={}, bla={} - {}.csv",
        "dataset_bugs_bla_chunk_pred": project_folder + data_folder + dataset_folder + "second/k={}/bla={}/pred/" + "{} _ k={}, bla={} - {}-pred.csv",
        "dataset_timeseries":          project_folder + data_folder + dataset_folder + dataset_file_name_ts       + ".csv"
    }
}


preprocessing_params = {
    "data": {
        "dataset_bugs_bla_chunk": {
            # time_stamp is the time that we change the class for a bug fixing.
            # id stands for identifier
            "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
            "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
        },
        "dataset_timeseries": {
            "columns_name":   ["class_id", "time_stamp", "label"],
            "columns_dtype" : {0: "int64", 1: "int64", 2:"int32"}
        }
    }
}

knn_model_params = {
    "train_size": 0.8,
    "n_neighbors": 5
}

progress_colours = ["webmaroon", "salmon", "orangered", "deeppink3", "crimson", 
                    "black", "blue", "gray", "cyan4", "darkgreen"]

In [11]:
mypaths["data"]["dataset_bugs_bla_chunk"] = mypaths["data"]["dataset_bugs_bla_chunk"].format(
    top_k,
    bug_localization_accuracy[0],
    dataset_file_name,
    top_k,
    bug_localization_accuracy[0],
    which_chunck
)

In [12]:
mypaths["data"]["dataset_bugs_bla_chunk_pred"] = mypaths["data"]["dataset_bugs_bla_chunk_pred"].format(
    top_k,
    bug_localization_accuracy[0],
    dataset_file_name,
    top_k,
    bug_localization_accuracy[0],
    which_chunck
)

In [13]:
print(mypaths["data"]["dataset_bugs_bla_chunk"])
print(mypaths["data"]["dataset_bugs_bla_chunk_pred"])

gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/CloudStack/second/k=20/bla=0.4/CloudStack_Bugs - v.01 _ k=20, bla=0.4 - 4.csv
gdrive/MyDrive/Colab Notebooks/paper/00- My Data/two-phased method/CloudStack/second/k=20/bla=0.4/pred/CloudStack_Bugs - v.01 _ k=20, bla=0.4 - 4-pred.csv


# Read Files

## df_timeseries

In [14]:
df_timeseries = pd.read_csv(
    mypaths["data"]["dataset_timeseries"], 
    names=preprocessing_params["data"]["dataset_timeseries"]["columns_name"], 
    dtype=preprocessing_params["data"]["dataset_timeseries"]["columns_dtype"], 
    header=None
)

## df_bugs_bla_chunk

In [15]:
df_bugs_bla_chunk = pd.read_csv(
    mypaths["data"]["dataset_bugs_bla_chunk"], 
    names=preprocessing_params["data"]["dataset_bugs_bla_chunk"]["columns_name"], 
    dtype=preprocessing_params["data"]["dataset_bugs_bla_chunk"]["columns_dtype"], 
    header=None
)

df_bugs_bla_chunk = [df_bugs_bla_chunk]
df_bugs_bla_chunk

[      bug_report_id  class_id  time_stamp  label
 0             14759     42844  1371707115      0
 1             14759     56900  1371707115      0
 2             14759     44013  1371707115      0
 3             14759     45337  1371707115      0
 4             14759     59821  1371707115      0
 ...             ...       ...         ...    ...
 3584          10272     44223  1511842335      0
 3585          10272     43981  1511842335      0
 3586          10275     38029  1511351614      0
 3587          10281     45947  1510893283      0
 3588          10289     42955  1510331649      0
 
 [3589 rows x 4 columns]]

# MyTimeSer

In [16]:
class MyTimeSer():
    
    def __init__(self, df_timeserAll, df_data, knn_model_params, threshold, bug_localization_accuracy):
        self.df_data_pred = df_data
        self.df_timeserAll = df_timeserAll
        self.accepted_num = {}
        self.threshold = threshold # [0.5, 0.5, ...]
        self.bla = bug_localization_accuracy
        self.knn_model_params = knn_model_params
        self.time_to_train = None
        
        
        for i, bla in enumerate(self.bla):
            self.accepted_num[bla] = {"isctss": 0}
            temp_df_data_pred = self.df_data_pred[i]
            
            for th in threshold:
                # add prediction columns
                new_column_name = self.prediction_column_name(th)
                temp_df_data_pred[new_column_name] = np.nan
                
                # initialize self.accepted_num
                # self.accepted_num[bla][th]: (iscts &) accup
                self.accepted_num[bla][th] = 0
    
    
    # ************************** helper ************************** #
    
    def prediction_column_name(self, th):
        return "predictions_" + str(th)
    
    
    # ************************** progress lines ************************** #
    
    def progress_lines(self, colours):
        desc = self.set_strings_to_equal_len_()
        total = len(self.df_data_pred[0])
        manager = enlighten.get_manager()
        progresses = []
        for i, bla in enumerate(self.bla):
            prog = manager.counter(total=total, desc=desc[i], unit="sample", color=colours[i])
            prog.refresh()
            progresses.append(prog)
        self.progresses = progresses
    
    
    def set_strings_to_equal_len_(self):
        words = []
        for bla in self.bla:
            desc = "train-test-" + str(bla)
            words.append(desc)
        
        max_len = 0
        longest_string_length = len(max(words, key=len))
        w = []
        for i, word in enumerate(words):
            temp = longest_string_length - len(word)
            w.append(word + " " * temp)
        return w
    
    
    # ************************** prediction ************************** #
    
    def make_predictions(self):
        for i, bla in enumerate(self.bla):
            # --- make prediction
            self.df_data_pred[i].apply(lambda row: self.flow_(row, i, bla), axis=1)

        # --- time to train
        totalSeconds = self.progresses[-1].elapsed
        hours = int(totalSeconds / 3600)
        minutes = int((totalSeconds - (hours * 3600)) / 60)
        seconds = int((totalSeconds - (hours * 3600 + minutes * 60)))
        self.time_to_train = f"{hours}h:{minutes}m:{seconds}s"
    
    
    def flow_(self, row, i, bla):
        # --- extract class time series.
        cts = self.extract_class_time_series_(row["class_id"], row["time_stamp"])
        
        # --- if cts is stationary, then make prediction & if accuracy pass the threshold, save the prediction
        is_cts_stationary = self.testSeries_(cts)
        if is_cts_stationary:
            self.accepted_num[bla]["isctss"] += 1
            accuracy, yhat = self.predictValue_(cts)
            for th in self.threshold:
                if accuracy > th:
                    self.accepted_num[bla][th] += 1
                    tempColumnName = self.prediction_column_name(th)
                    self.df_data_pred[i].loc[row.name, tempColumnName] = int(yhat)
        
        self.progresses[i].update()
    
    
    def extract_class_time_series_(self, class_id, bug_class_commitTime):
        df_class_time_series = self.df_timeserAll[
            (self.df_timeserAll["class_id"]  == class_id) & 
            (self.df_timeserAll["time_stamp"] < bug_class_commitTime)
        ]
        sorted_df_cts = df_class_time_series.sort_values(by="time_stamp", ascending=True)
        cts = sorted_df_cts["label"].to_numpy()
        return cts
    
    
    def testSeries_(self, cts):
        if len(cts) <= 10:
            return False
        
        result = adfuller(cts)
        critical_values = result[4].items()
        is_cts_stationary = True
        
        for alpha, critical_value in critical_values:
            ADF_Statistic = result[0]
            if critical_value < ADF_Statistic:
                is_cts_stationary = False
                break
        return is_cts_stationary
    
    
    def predictValue_(self, cts):
        max_accuracy = -1
        best_yhat = -1
        for query_length in range(3, 10):
            knnX, knnY = self.knnDS_(cts, query_length)
            query = np.array(cts[-query_length:])
            accuracy, yhat = self.knnPredict_(knnX, knnY, query)
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                best_yhat = yhat
        return (max_accuracy, best_yhat)
    
    
    def knnDS_(self, cts, ql):
        knnX = [cts[i: i + ql] for i in range(len(cts) - ql)] # [[0, -1, 1, 1, ...], [1, 0, 0, 0, ...]]
        knnY = [cts[i + ql]    for i in range(len(cts) - ql)] # [0, 1, ...]
        return knnX, knnY
    
    
    # x: [[0, -1, 1, 1, ...], [1, 0, 0, 0, ...]]
    # y: [0, 1, ...]
    def knnPredict_(self, x, y, query):
        # --- prepare train and test data
        size = int(len(x) * self.knn_model_params["train_size"])
        xtrain, xtest = x[0: size], x[size:]
        ytrain, ytest = y[0: size], y[size:]
        
        if len(xtrain) < 6:
            return 0, 0
        
        # --- train model
        knn = KNeighborsClassifier(n_neighbors=self.knn_model_params["n_neighbors"])
        knn.fit(xtrain, ytrain)
        
        # --- test the model
        accuracy = knn.score(xtest, ytest)
        
        # --- make prediction
        yhat = 0
        query = query.reshape(1, -1)
        yhat = knn.predict(query)
        return accuracy, yhat
    
    
    def free_memory(self):
        self.df_timeserAll = None

## obj

In [17]:
mt_data = MyTimeSer(
    df_timeseries, 
    df_bugs_bla_chunk, 
    knn_model_params, 
    knn_accuracy_threshold, 
    bug_localization_accuracy)

In [18]:
if knn_accuracy_threshold[0] == 1:
    temp_1 = df_bugs_bla_chunk[0].copy()
    temp_1["predictions_1"] = temp_1["label"].copy()
    mt_data.df_data_pred = [temp_1]
    mt_data.free_memory()
else:
    print("else")
    mt_data.progress_lines(progress_colours)
    mt_data.make_predictions()

else


In [19]:
mt_data.free_memory()

print(mt_data.time_to_train)
print(len(mt_data.df_data_pred))

mt_data.df_data_pred[0]

0h:1m:10s
1


,bug_report_id,class_id,time_stamp,label,predictions_0.5,predictions_0.55,predictions_0.6,predictions_0.65,predictions_0.7,predictions_0.75,predictions_0.8,predictions_0.85,predictions_0.9,predictions_0.95
0,14759,42844,1371707115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14759,56900,1371707115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14759,44013,1371707115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14759,45337,1371707115,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14759,59821,1371707115,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3584,10272,44223,1511842335,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3585,10272,43981,1511842335,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
3586,10275,38029,1511351614,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3587,10281,45947,1510893283,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


# save

In [20]:
df_bugs_bla_chunk[0].to_csv(mypaths["data"]["dataset_bugs_bla_chunk_pred"], index=False, na_rep="NULL")